# Libraries Loading

In [1]:
import numpy as np
import pandas as pd
import arabic
import pyarabic
import helpers
from helpers import Clean_data,separate_token_with_dicrites,save_h5
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from numpy import array, argmax
import os,errno,re,sys
import pickle
from itertools import product 
from pyarabic.araby import strip_tashkeel, strip_tatweel
import h5py
import random as rn
##########################
rn.seed(123456)
np.random.seed(123456)
input_data_path = "../data/Almoso3a_Alshe3rya/data/All_ksaied.csv"

In [2]:
'''
    * TESTING
'''
try:
    os.makedirs("data_testing/Encoded/8bits/WithoutTashkeel/Eliminated/")
    os.makedirs("data_testing/Encoded/8bits/WithoutTashkeel/Full_Data/")
    os.makedirs("data_testing/Encoded/8bits/WithTashkeel/Eliminated/")
    os.makedirs("data_testing/Encoded/8bits/WithTashkeel/Full_Data/")
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Can't create file for checkpoints or for logs please check ")
        raise
print("Paths created")

Paths created


# Data Loading

In [3]:
#input_data = pd.read_csv(input_data_path, index_col=0)
len(input_data)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


1831770

In [4]:
#input_data = pd.read_csv(input_data_path, index_col=0)

# TAHA
#input_data = sample

input_data.info(memory_usage='deep')
bahr = 'Bahr'
bayt = 'Bayt_Text'
#input_data  = input_data[[bayt, bahr]]
bahr = 'البحر'
bayt = 'البيت'
input_data  = input_data[[bayt, bahr]]
input_data.columns = ['Bayt_Text','Bahr']
our_alphabets = "".join(arabic.alphabet) + "".join(arabic.tashkeel)+" "
our_alphabets = "".join(our_alphabets)
input_data['Bayt_Text'] = input_data['Bayt_Text'].apply(lambda x: re.sub(r'[^'+our_alphabets+']','',str(x))).apply(lambda x: re.sub(r'  *'," ",x)).apply(lambda x: re.sub(r'ّ+', 'ّ', x)).apply(lambda x: x.strip())
input_data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1831770 entries, 0 to 1835252
Data columns (total 8 columns):
العصر           object
الشاعر          object
الديوان         object
القافية         object
البحر           object
الشطر الايسر    object
الشطر الايمن    object
البيت           object
dtypes: object(8)
memory usage: 1.8 GB


,Bayt_Text,Bahr
0,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا وَأَن تَ...,الطويل
1,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ وَلا سُرعَ...,الطويل
2,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً وَتَستَ...,الطويل
3,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ تُؤامِرُني ...,الطويل
4,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ وَأَفرَعَ...,الطويل
5,عَلى غَيرِ ذَنبٍ أَن أَكونَ جَنَيتُهُ سِوى قَو...,الطويل
6,لَعَمري لَنِعمَ المَرءُ تَدعو بِحَبلِهِ إِذا م...,الطويل
7,عَظيمُ رَمادِ القِدرِ لا مُتَعَبِّسٌ وَلا مُؤي...,الطويل
8,وَإِن صَرَّحَت كَحلٌ وَهَبَّت عَرِيَّةٌ مِنَ ا...,الطويل
9,صَبَرتُ عَلى وَطءِ المَوالي وَحَطمِهِم إِذا ضَ...,الطويل


In [5]:
# Taha: first class of the first observation
row_first_observation_class = input_data.iloc[0][1]
row_first_observation_class

'الطويل'

# Data Analysis

In [ ]:
# Taha: Counts the opservations of each class
#input_data.iloc[:, 1].value_counts()

# Removing non-classical meters

In [6]:
# Taha
# Very Expsensive Cell :(
# Output: 1 full_filtered_data, 2 eliminated_filtered_data
elminated_classic_bohor = ['الوافر', 'المنسرح','المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل',
                         'الرجز', 'الخفيف', 'البسيط']
full_bohor_classes = ['الوافر', 'المنسرح', 'المديد', 'المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل',
                         'الرجز', 'الخفيف', 'البسيط', 'المقتضب', 'الهزج', 'المضارع', 'المتدارك']

full_filtered_data = input_data.loc[input_data['Bahr'].isin(full_bohor_classes)]
full_filtered_data['Bayt_Text'] = full_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tatweel)
full_filtered_data['Bayt_Text'] = full_filtered_data['Bayt_Text'].apply(helpers.factor_shadda_tanwin)

eliminated_filtered_data = input_data.loc[input_data['Bahr'].isin(elminated_classic_bohor)]
eliminated_filtered_data['Bayt_Text'] = eliminated_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tatweel)
eliminated_filtered_data['Bayt_Text'] = eliminated_filtered_data['Bayt_Text'].apply(helpers.factor_shadda_tanwin)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value i

In [7]:
# Saving classsical meters only
# Expensive Cell
full_filtered_data.to_csv('full_filtered_data.csv', encoding='utf8')
eliminated_filtered_data.to_csv('eliminated_filtered_data.csv', encoding='utf8')

In [8]:
print(len(full_filtered_data))
print(len(eliminated_filtered_data))

1691671
1670177


# Take a Sample

In [9]:
# 100 bayt from every class
full_sample = pd.DataFrame(columns=full_filtered_data.columns)
for bahr in full_bohor_classes:
    XY = full_filtered_data[full_filtered_data.Bahr == bahr]
    first100 = XY.iloc[0:100]
    full_sample = full_sample.append(first100)
   
full_sample.to_csv('full_sample_100.csv', )

In [35]:
soso = pd.read_csv('full_sample_100.csv', index_col=0)
soso.head()
len(soso)
x = soso.Bayt_Text
print(type(x[0]))
print(len(x[0]))
x[0]
print(type(x))
set(x.apply(type))
soso[ soso.Bayt_Text.apply(type) == float]
#soso = soso.dropna()
#soso

soso.to_csv('full_sample_100', index=False)

<class 'str'>
76
<class 'pandas.core.series.Series'>


# >>>> Start Here.

In [40]:
# TO make the sample combatable with the rest of this notebook
full_filtered_data = full_sample
xx = pd.read_csv('full_sample_100.csv', index_col=0)
xx.head()
xx = xx.dropna()
xx.Bayt_Text.apply(len)
len(xx)
full_sample = xx
len(full_sample)

1599

In [41]:
eliminated_filtered_data = full_sample.copy()
full_filtered_data =  full_sample.copy()

# Encode Classes

In [42]:
'''
    Taha; Encoding Full Data classes as OneHot vectors. 
'''

full_data_label_encoder = LabelEncoder()
full_data_integer_encoded = full_data_label_encoder.fit_transform(list(full_filtered_data["Bahr"]))
# binary encode
full_data_onehot_encoder = OneHotEncoder(sparse=False)
full_data_integer_encoded = full_data_integer_encoded.reshape(len(full_data_integer_encoded), 1)
full_data_bohor_encoded = full_data_onehot_encoder.fit_transform(full_data_integer_encoded)
save_h5('data_testing/Encoded/8bits/WithoutTashkeel/Full_Data/full_data_Y_Meters.h5', 'Y', full_data_bohor_encoded)
save_h5('data_testing/Encoded/8bits/WithTashkeel/Full_Data/full_data_Y_Meters.h5', 'Y', full_data_bohor_encoded)
#saving with pickle: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
print('saving preprocessors....')

with open("data_testing/encoders_full_dat.pickle", 'wb') as pre_file:
    pickle.dump(full_data_label_encoder, pre_file, protocol=pickle.HIGHEST_PROTOCOL)
print('saved ....')

saving preprocessors....
saved ....


In [43]:
#print(type(full_data_label_encoder))

In [44]:
### testing
#print('loading saved encoders from %s'%(encoders_file_name))
with open("data_testing/encoders_full_dat.pickle", 'rb') as f:
    encoders = pickle.load(f)

##testing 
#inverted = encoders.inverse_transform([argmax(full_filtered_data["Bahr"][1,])])


# Taha
Ys = h5py.File('data_testing/Encoded/8bits/WithoutTashkeel/Full_Data/full_data_Y_Meters.h5', 'r')
class_of_first_observation = Ys['Y'][0]
decoded_first_class = encoders.inverse_transform([argmax( class_of_first_observation   )])

'''
if decoded_first_class == row_first_observation_class:
    print('NICE, Classes Encoded properly')
else:
    print('There is a problem in encoding-decoding classes.')
'''

"\nif decoded_first_class == row_first_observation_class:\n    print('NICE, Classes Encoded properly')\nelse:\n    print('There is a problem in encoding-decoding classes.')\n"

In [45]:
'''
    Taha; This cell is entirly doublicated.
    It Encodes the eliminated data classes.
'''
eliminated_data_label_encoder = LabelEncoder()
eliminated_data_integer_encoded = eliminated_data_label_encoder.fit_transform(list(eliminated_filtered_data["Bahr"]))
# binary encode
eliminated_data_onehot_encoder = OneHotEncoder(sparse=False)
eliminated_data_integer_encoded = eliminated_data_integer_encoded.reshape(len(eliminated_data_integer_encoded), 1)
eliminated_data_bohor_encoded = eliminated_data_onehot_encoder.fit_transform(eliminated_data_integer_encoded)
save_h5('data_testing/Encoded/8bits/WithoutTashkeel/Eliminated/Eliminated_data_Y_Meters.h5', 'Y', eliminated_data_bohor_encoded)
save_h5('data_testing/Encoded/8bits/WithTashkeel/Eliminated/Eliminated_data_Y_Meters.h5', 'Y', eliminated_data_bohor_encoded)
#saving with pickle: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
print('saving preprocessors....')
with open("data_testing/encoders_eliminated_data.pickle", 'wb') as pre_file:
    # pickle.dump(eliminated_data_bohor_encoded, pre_file, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(eliminated_data_label_encoder, pre_file, protocol=pickle.HIGHEST_PROTOCOL)
print('saved ...')

saving preprocessors....
saved ...


# Clean The Data

## WITH TASHKEEL

### FULL DATA

In [46]:
#bayt_len_full_withtashkeel = np.max((full_filtered_data.Bayt_Text.apply(separate_token_with_dicrites).apply(len)))
#separate_token_with_dicrites('123.213 الحمد لله')
#full_filtered_data.Bayt_Text.apply(separate_token_with_dicrites)

In [47]:
bayt_len_full_withtashkeel = np.max((full_filtered_data.Bayt_Text.apply(separate_token_with_dicrites).apply(len)))
print("bayt_len_full_withtashkeel done..")
full_filtered_data_with_tashkeel_cleaned = Clean_data(data_frame = full_filtered_data,max_bayt_len = bayt_len_full_withtashkeel,verse_column_name = 'Bayt_Text')
print("full_cleaned_data_with_tashkeel cleaned ..")
full_cleaned_data_with_tashkeel_encoded = full_filtered_data_with_tashkeel_cleaned.Bayt_Text.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, bayt_len_full_withtashkeel))
print("full_cleaned_data_with_tashkeel_encoded encoded ..")
full_cleaned_data_with_tashkeel_encoded_stacked = np.stack(full_cleaned_data_with_tashkeel_encoded,axis=0)
# save('../data/Encoded/8bits/WithTashkeel/Full_Data/full_data_matrix_with_tashkeel_8bitsEncoding.h5', 'X', full_cleaned_data_with_tashkeel_encoded_stacked) 
# Taha
save_h5('data_testing/Encoded/8bits/WithTashkeel/Full_Data/full_data_matrix_with_tashkeel_8bitsEncoding.h5', 'X', full_cleaned_data_with_tashkeel_encoded_stacked) 
print("full_cleaned_data_with_tashkeel_encoded saved ..")
print(full_cleaned_data_with_tashkeel_encoded_stacked.shape)

bayt_len_full_withtashkeel done..
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268


#### ELIMINATED DATA

In [48]:
bayt_len_eliminated_withtashkeel = np.max((eliminated_filtered_data.Bayt_Text.apply(separate_token_with_dicrites).apply(len)))
print("bayt_len_eliminated_withtashkeel done..")
eliminated_filtered_data_with_tashkeel_cleaned = Clean_data(data_frame = eliminated_filtered_data,max_bayt_len = bayt_len_eliminated_withtashkeel  ,verse_column_name = 'Bayt_Text')
print("eliminated_cleaned_data_with_tashkeel cleaned ..")
eliminated_cleaned_data_with_tashkeel_encoded = eliminated_filtered_data_with_tashkeel_cleaned.Bayt_Text.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, bayt_len_eliminated_withtashkeel))
print("eliminated_cleaned_data_with_tashkeel_encoded encoded ..")
eliminated_cleaned_data_with_tashkeel_encoded_stacked = np.stack(eliminated_cleaned_data_with_tashkeel_encoded,axis=0)
save_h5('data_testing/Encoded/8bits/WithTashkeel/Eliminated/eliminated_data_matrix_with_tashkeel_8bitsEncoding.h5', 'X', eliminated_cleaned_data_with_tashkeel_encoded_stacked) 
print("full_cleaned_data_with_tashkeel_encoded saved ..")
print(eliminated_cleaned_data_with_tashkeel_encoded_stacked.shape)
#encoded_byot = data_with_Tashkeel_cleaned.apply(lambda x: helpers.string_with_tashkeel_vectorizer_OneHot(x, maximum_bayt_len))

bayt_len_eliminated_withtashkeel done..
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
26

## WITHOUT TASHKEEL

### FULL DATA

In [49]:
full_filtered_data_without_tashkeel = full_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tashkeel)
bayt_len_full_without_tashkeel = np.max((full_filtered_data_without_tashkeel.apply(len)))
print("bayt_len_full_without_tashkeel done..")
full_cleaned_data_without_tashkeel_encoded = full_filtered_data_without_tashkeel.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, bayt_len_full_without_tashkeel))
print("full_filtered_data_without_tashkeel encoded ..")
full_filtered_data_without_tashkeel_staked = np.stack(full_cleaned_data_without_tashkeel_encoded,axis=0)
save_h5('data_testing/Encoded/8bits/WithoutTashkeel/Full_Data/full_data_matrix_without_tashkeel_8bitsEncoding.h5', 'X', full_filtered_data_without_tashkeel_staked) 
print("full_filtered_data_without_tashkeel_staked saved ..")
print(full_filtered_data_without_tashkeel_staked.shape)


bayt_len_full_without_tashkeel done..
full_filtered_data_without_tashkeel encoded ..
full_filtered_data_without_tashkeel_staked saved ..
(1599, 62, 8)


### ELIMINATE

In [50]:
eliminated_filtered_data_without_tashkeel = eliminated_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tashkeel)
bayt_len_eliminated_without_tashkeel = np.max((eliminated_filtered_data_without_tashkeel.apply(len)))
print("bayt_len_eliminated_without_tashkeel done..")
eliminated_cleaned_data_without_tashkeel_encoded = eliminated_filtered_data_without_tashkeel.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, bayt_len_eliminated_without_tashkeel))
print("eliminated_filtered_data_without_tashkeel encoded ..")
eliminated_filtered_data_without_tashkeel_staked = np.stack(eliminated_cleaned_data_without_tashkeel_encoded,axis=0)
save_h5('data_testing/Encoded/8bits/WithoutTashkeel/Eliminated/eliminated_data_matrix_without_tashkeel_8bitsEncoding.h5', 'X', full_filtered_data_without_tashkeel_staked) 
print("eliminated_filtered_data_without_tashkeel_staked saved ..")

print(eliminated_filtered_data_without_tashkeel_staked.shape)


bayt_len_eliminated_without_tashkeel done..
eliminated_filtered_data_without_tashkeel encoded ..
eliminated_filtered_data_without_tashkeel_staked saved ..
(1599, 62, 8)


# Trainging

In [ ]:
import Exp_Runner_test as exts

exts.runner(

)

'''
runner(encoded_x_data_path,
           encoded_y_data_path,
           test_size_param,
           num_layers_hidden,
           layers_type,
           validation_split_param,
           epochs_param,
           n_units,
           activation_output_function,
           load_weights_flag,
           last_or_max_val_acc,
           weighted_loss_flag,
           batch_size_param,
           earlystopping_patience,
           experiment_name,
           full_classes_encoder_path,
           eliminated_classes_encoder_path):
'''

In [ ]:
path='../checkpoints/Exp_1_eliminated_data_matrix_without_tashkeel_8bitsEncoding_LSTM_2_20_0'


In [ ]:
import re
import os

def remove_non_max(realtive_path):
    #re.sub('<.*?>', '', string)
    models = os.popen('find ' +  realtive_path + ' -maxdepth 1 -type f').readlines()
    tuples = []
    for i in range(len(models)):
        x = models[i][len(realtive_path):].strip('\n')
        y = re.sub('\S*-', '', x) 
        y = re.sub('\.hdf', '', y)
        try:
            y = float(y)
            tuples.append((x, y))
        except:
            continue

    sorted_tuples = sorted(tuples, reverse=True, key=lambda tup: tup[1])

    max_model = sorted_tuples[0]

    # rm the rest
    rmTheRestCommand = 'rm  '
    # File names starts with /
    for i in range(len(sorted_tuples)):
        if i == 0:
            continue
        fileName, _ = sorted_tuples[i]
        print(realtive_path + fileName)
        rmTheRestCommand += realtive_path + fileName + '  '


    # Executing the command:
    if os.system(rmTheRestCommand) == 0:
        print('Removing all checkpoints except the Last and the Max')
    else:
        print('Maybe there is no files other than the max and the Last!')

remove_non_max(path)

In [ ]:
rmTheRestCommand.split(' ')

In [ ]:
soso = pd.read_csv('full_sample_100.csv')

In [ ]:
print(soso.Bahr.unique())
print(len(soso.Bahr.unique()))

In [ ]:
dataX = h5py.File('data_testing/Encoded/8bits/WithoutTashkeel/Full_Data/full_data_matrix_without_tashkeel_8bitsEncoding.h5')

In [ ]:
dataX['X']

In [ ]:
dataY = h5py.File('data_testing/Encoded/8bits/WithoutTashkeel/Eliminated/Eliminated_data_Y_Meters.h5')

In [ ]:
dataY['Y']

In [52]:
import pickle

with open('data_testing/encoders_eliminated_data.pickle', 'rb') as f:
    encoders = pickle.load(f)
    print(type(encoders))

with open('data_testing/encoders_full_dat.pickle', 'rb') as f:
    encoders = pickle.load(f)
    print(type(encoders))

<class 'sklearn.preprocessing.label.LabelEncoder'>
<class 'sklearn.preprocessing.label.LabelEncoder'>


In [63]:
def checking_experiment_run(exp_name):
    '''
    this function is responsible for continue experiments running if running
    stop for any reson. 
    it will make file start from experiment that cut off.
    '''
    try:
        file = open('log.txt','r+')
    except:
        file = open('log.txt','w+')
    lines = {}
    line = file.readline()
    while line:
        parts = line.split(",")
        exp_n , exp_state = parts[0] , parts[1]
        lines[exp_n] = exp_state
        line = file.readline()
    if exp_name not in lines.keys():
        file.write(exp_name+","+"runing_0\n")
        file.close()
        return "added",0
    else:
        state, last_epoch = lines[exp_name].split('_')[0] , lines[exp_name].split('_')[1]
        return state , last_epoch.split('\n')[0]


In [65]:
checking_experiment_run('umar')

('runing', '0')

In [62]:
def update_log_file(exp_name,text,epoch_flage=False):
    
    def update(line,text,epoch_flage):
        if epoch_flage:
                line = line.split("_")[0]+"_"+text
        else:
                line = line = line.split(",")[0]+","+text
        return line
    try:
        lines = open("log.txt").read().split('\n')
        lines = [line if exp_name != line.split(",")[0] else update(line,text,epoch_flage) for line in lines]
        file = open('log.txt','w')
        file.write('\n'.join(lines))
        file.close()
        return True
    except:
        return False



In [70]:
update_log_file('umar', 'BODA', False)

True

In [71]:
!cat log.txt

umar,BODA


'=========='